In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys;src_dir = os.path.join(os.getcwd(), os.pardir, 'src');sys.path.append(src_dir)

In [2]:
import config
import pandas as pd
import ujson as json
from tqdm import tqdm

In [4]:
'''
LABELS:
1 = correct
2 = too far over the toes
3 = knocked knees
4 = back not straight
'''

def standard_session(name):
    return {
    'name': name,
    'squats-correct': [1,1,1],
    'squats-wrong': [2,3,4],
    'lunges-correct': [1,1,1],
    'lunges-wrong': [2,3,4],
    'sidelunges-correct': [1,1,1],
    'sidelunges-wrong': [4,4,4]
    }

evgeniya = standard_session('evgeniya')
evgeniya['lunges-correct'] = [1,1,1,1]

sergey = standard_session('sergey')
sergey['lunges-correct'] = [1,1,1,1]

tom = standard_session('tom')
tom['sidelunges-correct'] = [1,1,1,1]

tim = standard_session('tim')
tim['lunges-wrong'] = [2,3,4,4]

jessa = standard_session('jessa')

pedro = standard_session('pedro')

stella = standard_session('stella')

toon = standard_session('toon')
toon['squats-correct'] = [1,1,1,1]

gust = standard_session('gust')

vincent = standard_session('vincent')
vincent['sidelunges-wrong'] = [4,4,4,4]

persons = [evgeniya,sergey,tom,tim,jessa,pedro,stella,toon,gust,vincent]

def get_person(personfolder):
    for person in persons:
        if person['name'] in personfolder:
            return person

def validxeffile(x):
    minsizemb = 500
    minfilesize = minsizemb * 1000 * 1000
    return '.xef' in x and os.path.getsize(x) > minfilesize

def read_jointsjson(filename):
    with open(filename,'rb') as f:
        bodies = json.load(f)
    jointsframes = [x['Joints'] if x else None for x in bodies]
    data = []
    for jointsframe in jointsframes:
        jointsdict = {}
        if jointsframe is not None:
            for k,v in jointsframe.items():
                for coor in ['X','Y','Z']:
                    jointsdict[k+coor] = v['Position'][coor]
        data.append(jointsdict)
    
    df = pd.DataFrame(data)
    df = df.fillna(method='pad').fillna(method='backfill')
    return df

jsonfiles = []
df_id = 1
overview = []
for personfolder in os.listdir(config.frontviewjson):
    for exercise in ['squat','lunge','sidelunge']:
        for correct in ['correct','wrong']:
            person = get_person(personfolder)
            exercise_set = exercise + "s-" + correct
            ei_folder = "/".join([config.frontviewjson,personfolder,exercise_set])
            eis = [x for x in os.listdir(ei_folder) if validxeffile(ei_folder + "/" + x)]
            ets = person[exercise_set]
            assert(len(eis) == len(ets))
            for exer_inst,exec_type in zip(eis,ets):
                jsonfile = ei_folder + "/" + exer_inst.replace(".xef","-bodyframes.json")
                jsonfiles.append(jsonfile)

In [9]:
import data.split as split
dfs = []
# proxy for timing extracting data
for f in tqdm(jsonfiles):
    df = read_jointsjson(f)
    dfs.append(df)
# timing splitting data
for f in tqdm(dfs):
    split.split(df)


100%|██████████| 186/186 [00:05<00:00, 33.17it/s]


In [10]:
import features.angles as ang

# timing constructing feature vectors
for df in tqdm(dfs):
    ang = ang.kinect.get_angles(df)
    # computing statistics is negligible in time cost

100%|██████████| 186/186 [00:09<00:00, 19.31it/s]


In [24]:
# timing classifying examples
import pickle
import numpy as np
with open('../data/resultsQ1.pkl','rb') as fh:
    results = pickle.load(fh)

for result in results:
    if "XG" in result[2]:
        print(np.mean(result[3]["score_time"])*10/1790*10)

6.56396983056095e-05
6.426246472577143e-05
5.3541620350416816e-05
5.1069526033028546e-05
0.00013293500719123712
0.00013011394266309684
